# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.14108


extracting tarball to tmp_2302.14108...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.14137


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.14108/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.14788


extracting tarball to tmp_2302.14788...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.14800


extracting tarball to tmp_2302.14800...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14137-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14137) | **Detailed Analysis of the TeV γ-Ray Sources 3HWC J1928+178, 3HWC  J1930+188, and the New Source HAWC J1932+192**  |
|| A. Albert, et al. -- incl., <mark>W. H. Lee</mark> |
|*Appeared on*| *2023-03-01*|
|*Comments*| **|
|**Abstract**| The latest High Altitude Water Cherenkov (HAWC) point-like source catalog up to 56 TeV reported the detection of two sources in the region of the Galactic plane at galactic longitude 52\deg < l < 55\deg, 3HWC J1930+188 and 3HWC J1928+178. The first one is associated with a known TeV source, the supernova remnant SNR G054.1+00.3. It was discovered by one of the currently operating Imaging Atmospheric Cherenkov Telescope (IACT), the Very Energetic Radiation Imaging Telescope Array System (VERITAS), detected by the High Energy Stereoscopic System (H.E.S.S.), and identified as a composite SNR. However, the source 3HWC J1928+178, discovered by HAWC and coincident with the pulsar PSR J1928+1746, was not detected by any IACT despite their long exposure on the region, until a recent new analysis of H.E.S.S. data was able to confirm it. Moreover, no X-ray counterpart has been detected from this pulsar. We present a multicomponent fit of this region using the latest HAWC data. This reveals an additional new source, HAWC J1932+192, which is potentially associated with the pulsar PSR J1932+1916, whose gamma-ray emission could come from the acceleration of particles in its pulsar wind nebula. In the case of 3HWC J1928+178, several possible explanations are explored, in a attempt to unveil the origins of the very-high-energy gamma-ray emission. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14108-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14108) | **Beyond Ultra-Diffuse Galaxies II: Environmental Quenching of Mass-Size  Outliers Among the Satellites of Milky Way Analogs**  |
|| <mark>Jiaxuan Li</mark>, et al. |
|*Appeared on*| *2023-03-01*|
|*Comments*| *13 pages, 6 figures, submitted to ApJ. Key results are shown in Figures 4-6*|
|**Abstract**| Recent observations have reignited interest in a population of dwarf galaxies that are large and diffuse for their mass, often called ultra-diffuse galaxies. However, the origin and evolution of these mass-size outliers and the role of environment are still unclear. Using the exquisitely deep and wide Hyper Suprime-Cam Strategic Survey images, we search for ultra-puffy galaxies, defined as being $1.5\sigma$ larger than the average size for their mass, around Milky-Way--like galaxies. We present the size and radial distributions of mass-size outliers, and derive their quenched fraction to explore the impact of environment. Surprisingly, despite being outliers in size, the ultra-puffy galaxies have a similar quenched fraction as normal-sized satellites of Milky Way analogs in both observations and simulations, suggesting that quenching is not tied to being a mass-size outlier. The quenched fraction is higher for the ultra-puffy galaxies associated with redder hosts as well as those that are closer to the host in projection. In contrast, the ultra-diffuse galaxies are overall redder and more quiescent compared with normal satellites. We show that the classic definition of ultra-diffuse galaxies is heavily weighted towards quenched galaxies and thus cannot be used for a study of quenching of mass-size outliers. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14185-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14185) | **Low-Frequency Radio Recombination Lines Away From the Inner Galactic  Plane**  |
|| Akshatha K. Vydula, et al. -- incl., <mark>David Lewis</mark> |
|*Appeared on*| *2023-03-01*|
|*Comments*| *23 pages, 13 figures, 2 tables, submitted to AAS*|
|**Abstract**| Diffuse radio recombination lines (RRLs) in the Galaxy are possible foregrounds for redshifted 21~cm experiments. We use EDGES drift scans centered at $-26.7^{\circ}$~declination to characterize diffuse RRLs across the southern sky. We find RRLs averaged over the large antenna beam ($72^{\circ} \times 110^{\circ}$) reach minimum amplitudes between right ascensions~2-6~h. In this region, the C$\alpha$ absorption amplitude is $33\pm11$~mK (1$\sigma$) averaged over 50-87~MHz ($27\gtrsim z \gtrsim15$ for the 21~cm line) and increases strongly as frequency decreases. C$\beta$ and H$\alpha$ lines are consistent with no detection with amplitudes of $13\pm14$ and $12\pm10$~mK (1$\sigma$), respectively. For 108-124.5~MHz ($z\approx11$), we find no evidence for hydrogen or carbon lines at the noise level of 3.4~mK (1$\sigma$). Conservatively assuming observed lines come broadly from the diffuse interstellar medium, as opposed to a few specific regions, these amplitudes provide upper limits on the intrinsic diffuse lines. The observations support expectations that Galactic RRLs can be neglected as significant foregrounds for a large region of sky until redshifted 21~cm experiments, particularly those targeting Cosmic Dawn, move beyond the detection phase. For C$\alpha$, we find a best-fit departure coefficient product, $b_n \beta_n$, that varies smoothly as a function of frequency between $0.79<b_n\beta_n<4.5$ along the inner Galactic Plane and between $0<b_n\beta_n<2.3$ away from the inner Galactic Plane, yields better agreement between model and data than assuming local thermodynamic equilibrium conditions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14782-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14782) | **Deep Synoptic Array science: A massive elliptical host among two  galaxy-cluster fast radio bursts**  |
|| <mark>Kritti Sharma</mark>, et al. |
|*Appeared on*| *2023-03-01*|
|*Comments*| *18 pages, 9 figures*|
|**Abstract**| The stellar population environments associated with fast radio burst (FRB) sources provide important insights for developing their progenitor theories. We expand the diversity of known FRB host environments by reporting two FRBs in massive galaxy clusters discovered by the Deep Synoptic Array (DSA-110) during its commissioning observations. FRB 20220914A has been localized to a star-forming, late-type galaxy at a redshift of 0.1139 with multiple starbursts at lookback times less than $\sim$3.5 Gyr in the Abell 2310 galaxy cluster. Although the host galaxy of FRB 20220914A is similar to typical FRB hosts, the FRB 20220509G host stands out as a quiescent, early-type galaxy at a redshift of 0.0894 in the Abell 2311 galaxy cluster. The discovery of FRBs in both late and early-type galaxies adds to the body of evidence that the FRB sources have multiple formation channels. Therefore, even though FRB hosts are typically star-forming, there must exist formation channels consistent with old stellar population in galaxies. The varied star formation histories of the two FRB hosts we report indicate a wide delay-time distribution of FRB progenitors. Future work in constraining the FRB delay-time distribution, using methods we develop herein, will prove crucial in determining the evolutionary histories of FRB sources. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14783-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14783) | **Probing the rapid formation of black holes and their galaxy hosts in  QSOs**  |
|| Karla Alejandra Cutiva-Alvarez, Roger Coziol, <mark>Juan Pablo Torres-Papaqui</mark> |
|*Appeared on*| *2023-03-01*|
|*Comments*| *Accepted in MNRAS 2023 February 28. Received 2023 February 28; in original form 2022 July 09*|
|**Abstract**| Using the modelling code X-CIGALE, we reproduced the SEDs of 1,359 SDSS QSOs within the redshift range 0 < z < 4, for which we have NIR/MIR fluxes with the highest quality and spectral data characterizing their SMBHs. Consistent with a rapid formation of the host galaxies, the star formation histories (SFHs) have small e-folding, at most 750 Myrs using an SFH function for Spiral or 1000 Myrs using one for Elliptical. Above z \sim 1.6, the two solutions are degenerate, the SEDs being dominated by the AGN continuum and high star formation rates (SFRs), typical of starburst galaxies, while at lower redshifts the starburst nature of the host, independent from its morphology, is better reproduced by an Spiral SFH. In general, the SFR increases with the redshift, the mass of the bulge, the AGN luminosity and Eddington ratio, suggesting there is no evidence of AGN quenching of star formation. Comparing the specific BHAR with specific SFR, all the QSOs at any redshift trace a linear sequence below the Eddington luminosity, in parallel and above the one-to-one relation, implying that QSOs are in a special phase of evolution during which the growth in mass of their SMBH is more rapid than the growth in mass of their galaxy hosts. This particular phase is consistent with a scenario where the galaxy hosts of QSOs in the past grew in mass more rapidly than their SMBHs, suggesting that a high star formation efficiency during their formation was responsible in limiting their masses |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14788-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14788) | **Deep Synoptic Array science: Two fast radio burst sources in massive  galaxy clusters**  |
|| Liam Connor, et al. -- incl., <mark>Kritti Sharma</mark> |
|*Appeared on*| *2023-03-01*|
|*Comments*| **|
|**Abstract**| The hot gas that constitutes the intracluster medium (ICM) has been studied at X-ray and millimeter/sub-millimeter wavelengths (Sunyaev-Zeldovich effect) for decades. Fast radio bursts (FRBs) offer an additional method of directly measuring the ICM and gas surrounding clusters, via observables such as dispersion measure (DM) and Faraday rotation measure (RM). We report the discovery of two FRB sources detected with the Deep Synoptic Array (DSA-110) whose host galaxies belong to massive galaxy clusters. In both cases, the FRBs exhibit excess extragalactic DM, some of which likely originates in the ICM of their respective clusters. FRB 20220914A resides in the galaxy cluster Abell 2310 at z=0.1125 with a projected offset from the cluster center of 520 kpc. The host of a second source, FRB 20220509G, is an elliptical galaxy at z=0.0894 that belongs to the galaxy cluster Abell 2311 at projected offset of 870 kpc. These sources represent the first time an FRB has been localized to a galaxy cluster. We combine our FRB data with archival X-ray, SZ, and optical observations of these clusters in order to infer properties of the ICM, including a measurement of gas temperature from DM and ySZ of 0.8-3.9 keV. We then compare our results to massive cluster halos from the IllustrisTNG simulation. Finally, we describe how large samples of localized FRBs from future surveys will constrain the ICM, particularly beyond the virial radius of clusters. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14800-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14800) | **The JCMT Nearby Galaxies Legacy Survey: SCUBA-2 observations of nearby  galaxies**  |
|| Kate Pattle, <mark>Walter Gear</mark>, Christine D. Wilson |
|*Appeared on*| *2023-03-01*|
|*Comments*| *Accepted for publication in MNRAS. 15 pages, 12 figures, 8 tables (plus 15 pages of appendices, with 31 figures)*|
|**Abstract**| We present 850$\mu$m observations of a sample of 8 nearby spiral galaxies, made using the SCUBA-2 camera on the James Clerk Maxwell Telescope (JCMT) as part of the JCMT Nearby Galaxies Legacy Survey (NGLS). We corrected our data for the presence of the $^{12}$CO $J=3\to 2$ line in the SCUBA-2 850$\mu$m bandwidth using NGLS HARP data, finding a typical $^{12}$CO contribution of $\sim 20$%. We measured dust column densities, temperatures and opacity indices by fitting spectral energy distributions constructed from SCUBA-2 and archival Herschel observations, and used archival GALEX and Spitzer data to make maps of surface density of star formation ($\Sigma_{\rm SFR}$). Typically, comparing SCUBA-2-derived H$_2$ surface densities ($\Sigma_{\rm H_2}$) to $\Sigma_{\rm SFR}$ gives shallow star formation law indices within galaxies, with SCUBA-2-derived values typically being sublinear and Herschel-derived values typically being broadly linear. This difference is likely due to the effects of atmospheric filtering on the SCUBA-2 data. Comparing the mean values of $\Sigma_{\rm H_2}$ and $\Sigma_{\rm SFR}$ of the galaxies in our sample returns a steeper star formation law index, broadly consistent with both the Kennicutt-Schmidt value of 1.4 and linearity. Our results show that a SCUBA-2 detection is a good predictor of star formation. We suggest that Herschel emission traces gas in regions which will form stars on timescales $\sim 5-100$ Myr, comparable to the star formation timescale traced by GALEX and Spitzer data, while SCUBA-2 preferentially traces the densest gas within these regions, which likely forms stars on shorter timescales. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14155-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14155) | **Discovery of a quiescent galaxy at z=7.3**  |
|| Tobias J. Looser, et al. -- incl., <mark>Anna de Graaff</mark>, <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2023-03-01*|
|*Comments*| **|
|**Abstract**| Local galaxies are known to broadly follow a bimodal distribution: actively star forming and quiescent system (i.e. galaxies with no or negligible star formation activity at the epoch of observation). Why, when and how such bimodality was established, and whether it has been associated with different processes at different cosmic epochs, is still a key open question in extragalactic astrophysics. Directly observing early quiescent galaxies in the primordial Universe is therefore of utmost importance to constraining models of galaxy formation and transformation. Early quiescent galaxies have been identified out to redshift $z < 5$, and these are all found to be massive ($M_{*}>10^{10}~M_{\odot}$). Here we report the discovery of a quiescent galaxy at z$=$7.3, when the Universe was only 700 Myr old - about 5% of its current age. The JWST/NIRSpec spectrum of this galaxy from our JADES programme exhibits a complete absence of nebular emission lines, while the Balmer break and Ly$\alpha$ drop are unambiguously detected. We infer that this galaxy experienced a short and intense burst of star formation followed by rapid quenching, about 10-20 Myr before the epoch of observation. Particularly interesting is that the mass of this quiescent galaxy is only $\sim$4-6$\times 10^8~M_{\odot}$. This mass range is sensitive to various feedback mechanisms that can result in temporary or permanent quiescence. Therefore this galaxy represents a unique opportunity to learn more about galaxy formation and transformation in the early Universe. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error expected string or bytes-like object</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.14137.md
    + _build/html/tmp_2302.14137/./summary_initial.png
    + _build/html/tmp_2302.14137/./HAWC_CO_velocity.png
    + _build/html/tmp_2302.14137/./Veritas_Chandra_NuSTAR.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Detailed Analysis of the TeV $\gamma$-Ray Sources 3HWC J1928+178, 3HWC J1930+188, and the New Source HAWC J1932+192

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.14137-b31b1b.svg)](https://arxiv.org/abs/2302.14137)<mark>Appeared on: 2023-03-01</mark> - __

</div>
<div id="authors">

A. Albert, et al. -- incl., <mark><mark>W. H. Lee</mark></mark>

</div>
<div id="abstract">

**Abstract:** The latest High Altitude Water Cherenkov (HAWC) point-like source catalog up to  56 TeV reported the detection of two sources in the region of the Galactic plane at galactic longitude 52 \textdegree $< \ell <$ 55 \textdegree , 3HWC J1930+188 and 3HWC J1928+178. The first one is associated with a known TeV source, the supernova remnant SNR G054.1+00.3. It was discovered by one of the currently operating Imaging Atmospheric Cherenkov Telescope (IACT), the Very Energetic Radiation Imaging Telescope Array System (VERITAS), detected by the High Energy Stereoscopic System (H.E.S.S.), and identified as a composite SNR.However, the source 3HWC J1928+178, discovered by HAWC and coincident with the pulsar PSR J1928+1746, was not detected by any IACT despite their long exposure on the region, until a recent new analysis of H.E.S.S. data was able to confirm it.Moreover, no X-ray counterpart has been detected from this pulsar.We present a multicomponent fit of this region using the latest HAWC data. This reveals an additional new source, HAWC J1932+192, which is potentially associated with the pulsar PSR J1932+1916, whose $\gamma$ -ray emission could come from the acceleration of particles in its pulsar wind nebula. In the case of 3HWC J1928+178, several possible explanations are explored, in a attempt to unveil the origins of the very-high-energy $\gamma$ -ray emission.

</div>

<div id="div_fig1">

<img src="tmp_2302.14137/./summary_initial.png" alt="Fig3" width="100%"/>

**Figure 3. -**  The significance map (a) indicates the region of interest (ROI) of radius 3\textdegree5 (the white circle) and the two components for J1928 and J1930. The blue/green dot and circle show the initial/fitted position and size. The width of the green circle represents the 1$\sigma$ uncertainty on the size of the Gaussian.
    Map (b) is the significance map of the model in the ROI.
    Map (c) is the significance map of the residuals in the ROI and the significance distribution in the inner 2\textdegree  radius region, with a Gaussian fit. The color scale holds for all maps.
     (*summary_a*)

</div>
<div id="div_fig2">

<img src="tmp_2302.14137/./HAWC_CO_velocity.png" alt="Fig8" width="100%"/>

**Figure 8. -** The HAWC significance map is shown in panel (a). The black circle shows the location and 1$\sigma$ uncertainty of the HAWC source \citep{3HWC_catalog}. The black cross is the location of the pulsar PSR J1928+1746. The white box is the region with $>5\sigma$$\gamma$-ray emission of 3HWC J1928+178, with the velocity dispersion being averaged and plotted in panel (b) as a function of the brightness temperature. For the three peaks at $\sim$4.5 km s$^{-1}$, $\sim$22 km s$^{-1}$ and $\sim$46 km s$^{-1}$, the $^{13}$CO maps are shown in panel (c). (*HAWC_CO_velocity*)

</div>
<div id="div_fig3">

<img src="tmp_2302.14137/./Veritas_Chandra_NuSTAR.png" alt="Fig1" width="100%"/>

**Figure 1. -** Multiwavelength view of the region surrounding 3HWC J1928+177. The middle map is the VERITAS excess map of the region, adapted from \citet{Veritas_Fermi_2HWCsources}. Superimposed are the locations and the 1$\sigma$ uncertainties on the locations of the HAWC sources (blue circles) and the Fermi 4FGL sources (green circles), as well as the locations of the pulsars (red crosses). The white contours are HAWC significance contours for 5$\sigma$, 6$\sigma$, 7$\sigma$, 8$\sigma$, 10$\sigma$ and 12$\sigma$ for 1523 days of data.
    The top source, 3HWC J1930+188, is detailed in the zoomed-in view on the left-hand side. The locations of the counterparts detected by VERITAS (yellow) and H.E.S.S. (pink) are represented. The extension of the radio emission is also shown (cyan). The dashed white box represents the size of the composite image at the top (3$\arcmin$ - 0\textdegree05). It depicts the X-ray emission of the pulsar (the bright white star) and the PWN detected by _Chandra_(blue - NASA/CXC/SAO/T.Temim et al.), as well as the IR emission detected by _Spitzer_(green is 8$\mu$m and red is 24$\mu$m - NASA/JPL-Caltech), revealing the dusty remains of a collapsed star. The bottom source 3HWC J1928+178 is detailed in the zoomed-in image on the right-hand side. The dashed black box represents the NuSTAR background-subtracted map shown at the bottom (adapted from \citet{J1928_dark_accelerator}). The bright source to the bottom right is CXO J192812.0+174712.
     (*Veritas_Chandra_NuSTAR*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

215  publications in the last 7 days.
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.
